In [1]:
import numpy as np 
import pandas as pd 

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

import pickle
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/usr/local/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
Combined_data = pd.read_csv('LosAngeles_2022.csv')
# Combined_data['last_review'] = pd.to_datetime(Combined_data['last_review'], infer_datetime_format=True) 
Combined_data.drop(['host_id', 'id', 'host_name','name',
                    'last_review', 'neighbourhood', 'license', 'number_of_reviews_ltm'], 
                   axis=1, inplace=True)

# fill NAs
Combined_data['reviews_per_month'] = Combined_data['reviews_per_month'].fillna(0)
Combined_data['neighbourhood_group'] = Combined_data['neighbourhood_group'].fillna('unknown')

# remove outliers and log transformation
Combined_data = Combined_data[np.log1p(Combined_data['price']) < 8]
Combined_data = Combined_data[np.log1p(Combined_data['price']) > 3]
Combined_data['price'] = np.log1p(Combined_data['price'])
Combined_data['reviews_per_month'] = Combined_data[Combined_data['reviews_per_month'] < 17.5]['reviews_per_month']
Combined_data['reviews_per_month'] = Combined_data['reviews_per_month'].fillna(0)
Combined_data['minimum_nights'] = np.log1p(Combined_data['minimum_nights'])

# segment numeric variable
Combined_data['all_year_avail'] = 1*(Combined_data['availability_365']>353)
Combined_data['low_avail'] = 1*(Combined_data['availability_365']< 12)
Combined_data['no_reviews'] = 1*(Combined_data['reviews_per_month']==0)

# Combined_data['room_type'] = Combined_data['room_type'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))
# Combined_data['room_type'] = (Combined_data['room_type']).str.replace(' ', '_')
print(np.unique(Combined_data['room_type']))

print(Combined_data.shape)

# one hot encode categorical variables
categorical_features = Combined_data.select_dtypes(include=['object'])
print(categorical_features.columns)
print(categorical_features.shape)
categorical_features_one_hot = pd.get_dummies(categorical_features)

# select numerical variables
numerical_features =  Combined_data.select_dtypes(exclude=['object'])
print(numerical_features.columns)
print(numerical_features.shape)

y = numerical_features.price
numerical_features = numerical_features.drop(['price'], axis=1)

X = np.concatenate((numerical_features, categorical_features_one_hot), axis=1) # no column names
X_df = pd.concat([numerical_features, categorical_features_one_hot], axis=1) # with column names

print(X_df.shape)
print(X_df.columns)
# Processed_data = pd.concat([X_df, y], axis = 1)
# Processed_data.to_csv('Airbnb_LA_Processed.dat')


['Entire home/apt' 'Hotel room' 'Private room' 'Shared room']
(41597, 13)
Index(['neighbourhood_group', 'room_type'], dtype='object')
(41597, 2)
Index(['latitude', 'longitude', 'price', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'all_year_avail', 'low_avail', 'no_reviews'],
      dtype='object')
(41597, 11)
(41597, 18)
Index(['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'all_year_avail', 'low_avail', 'no_reviews',
       'neighbourhood_group_City of Los Angeles',
       'neighbourhood_group_Other Cities',
       'neighbourhood_group_Unincorporated Areas',
       'neighbourhood_group_unknown', 'room_type_Entire home/apt',
       'room_type_Hotel room', 'room_type_Private room',
       'room_type_Shared room'],
      dtype='object')


In [3]:
categorical_features_one_hot
numerical_features

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,all_year_avail,low_avail,no_reviews
0,33.983010,-118.386070,3.433987,2,0.02,1,43,0,0,0
1,34.096250,-118.346050,3.433987,37,0.38,1,230,0,0,0
2,34.106320,-118.223610,3.433987,27,0.20,1,329,0,0,0
3,34.004400,-118.480950,2.079442,23,0.17,2,329,0,0,0
4,34.095740,-118.277880,3.433987,154,1.09,4,131,0,0,0
...,...,...,...,...,...,...,...,...,...,...
42035,33.623209,-117.918492,3.433987,0,0.00,1,365,1,0,1
42036,33.764591,-117.992373,3.465736,0,0.00,1,364,1,0,1
42037,33.742780,-117.942232,0.693147,0,0.00,45,364,1,0,1
42038,33.744660,-117.941931,0.693147,0,0.00,45,364,1,0,1


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Dimensions of the training feature matrix: {}'.format(X_train.shape))
print('Dimensions of the training target vector: {}'.format(y_train.shape))
print('Dimensions of the test feature matrix: {}'.format(X_test.shape))
print('Dimensions of the test target vector: {}'.format(y_test.shape))

Dimensions of the training feature matrix: (33277, 18)
Dimensions of the training target vector: (33277,)
Dimensions of the test feature matrix: (8320, 18)
Dimensions of the test target vector: (8320,)


In [5]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train.shape

(33277, 18)

In [6]:
with open('./pickles/scaler', 'wb') as file:
  pickle.dump(scaler, file)

In [7]:
# aim to report RMSE metric over 5-fold cross validation
n_folds = 5

# rmse
def rmse_cv(model, X_train = X_train):
    kf = KFold(n_folds, shuffle=True, random_state=2022).get_n_splits(numerical_features)
    return cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=kf)


In [8]:
xbgreg_best = XGBRegressor(n_estimators=1000, #learning_rate=0.1, 
                           early_stopping=5, max_depth=9, min_child_weight=5)
xbgreg_CV_best = -rmse_cv(xbgreg_best)
xbgreg_best.fit(X_train, y_train) 
y_train_xgbreg = xbgreg_best.predict(X_train)
y_test_xgbreg = xbgreg_best.predict(X_test)
xgb_best_results = pd.DataFrame({'algorithm':['XGBRegressor'],
                                 'CV error': xbgreg_CV_best.mean(), 
                                 'CV std': xbgreg_CV_best.std(),
                                 'training error': [mean_squared_error(y_train, y_train_xgbreg)],
                                 'test error': [mean_squared_error(y_test, y_test_xgbreg)],
                                 'training_r2_score': [r2_score(y_train, y_train_xgbreg)],
                                 'test_r2_score': [r2_score(y_test, y_test_xgbreg)]})
xgb_best_results

[15:15:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "early_stopping" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[15:15:53] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "early_stopping" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[15:16:13] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "early_stopping" } might not be used.

  This could be a false alarm, with some parameters getting used by language 

,algorithm,CV error,CV std,training error,test error,training_r2_score,test_r2_score
0,XGBRegressor,0.275854,0.005322,0.03994,0.270161,0.946719,0.637325


In [9]:
with open('./pickles/model', 'wb') as file:
  pickle.dump(xbgreg_best, file)